# build dataset

eric gagliano   
jan 17 2024  
notebook to build datasets
  * data sources
    * for Sentinel-1 RTC data: https://planetarycomputer.microsoft.com/dataset/sentinel-1-rtc 
    * LIA: https://github.com/egagli/generate_sentinel1_local_incidence_angle_maps
    * LC: https://planetarycomputer.microsoft.com/dataset/esa-worldcover
    * FCF: https://zenodo.org/records/3939050
    * Snow classification: https://nsidc.org/data/nsidc-0768/versions/1
    * Snow cover (snow / no snow): https://nsidc.org/data/mod10a1f/versions/61
        * previously tried https://planetarycomputer.microsoft.com/dataset/modis-10A1-061 (or 8 day)

In [1]:
!pip -q install earthaccess

In [2]:
import geopandas as gpd
import create_dataset
import numpy as np

In [3]:
shapefile_folder = 'data/shapefiles/'

geojson = 'iceland_trollaskagi_peninsula.geojson'  
geojson = 'grandmesa.geojson'
geojson = 'banded_peak_ranch.geojson'

fn = f'{shapefile_folder}{geojson}'

bbox_gdf = gpd.read_file(fn)
bbox_gdf.attrs = {'filename':fn}

In [4]:
resolution = 30

In [5]:
year = 2020

In [6]:
time_reference_scenes = slice(f'{year}-01-01',f'{year}-03-31') # northern hemisphere winter winter
time_reference_scenes = slice(f'{year}-06-01',f'{year}-07-31') 

time_target_scene = f'{year}-06-27' # try and support time_wet_scene_asc and time_wet_scene_desc
time_target_scene = f'{year}-04-27' # try and support time_wet_scene_asc and time_wet_scene_desc

In [7]:
reference_scenes_aggregation_technique = 'median' # mean / median / max

In [8]:
%%time
dataset = create_dataset.create_dataset(bbox_gdf, 
                                        year, 
                                        time_reference_scenes, 
                                        time_target_scene, 
                                        reference_scenes_aggregation_technique, 
                                        resolution)

/srv/conda/envs/notebook/lib/python3.11/site-packages/pystac_client/item_search.py:826: FutureWarning: get_items() is deprecated, use items() instead
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.11/site-packages/xarray/core/indexing.py:1443: PerformanceWarning: Slicing with an out-of-order index is generating 29 times more chunks
  return self.array[key]
/srv/conda/envs/notebook/lib/python3.11/site-packages/pystac_client/item_search.py:826: FutureWarning: get_items() is deprecated, use items() instead
  warnings.warn(


Creating new LIA data.


/srv/conda/envs/notebook/lib/python3.11/site-packages/stackstac/prepare.py:363: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


Creating output file that is 1663P x 2260L.
Processing /tmp/dems/dem.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Returned 664 Items


/srv/conda/envs/notebook/lib/python3.11/site-packages/stackstac/prepare.py:363: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


...

Granules found: 1
 Getting 1 granules, approx download size: 0.0 GB


QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 58.8 s, sys: 13.5 s, total: 1min 12s
Wall time: 2min 10s


In [ ]:
%%time
dataset.to_zarr(f'data/datasets/{geojson.split(".")[0]}_{year}_tar_{time_target_scene}_ref_{time_reference_scenes.start}_{time_reference_scenes.stop}_{reference_scenes_aggregation_technique}_{resolution}.zarr',mode='w')

/srv/conda/envs/notebook/lib/python3.11/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(


In [10]:
%%time
dataframe = create_dataset.dataset_to_dataframe(dataset)

CPU times: user 5min 54s, sys: 2min 4s, total: 7min 58s
Wall time: 2min 27s


In [11]:
%%time
dataframe.to_parquet(f'data/dataframes/{geojson.split(".")[0]}_{year}_tar_{time_target_scene}_ref_{time_reference_scenes.start}_{time_reference_scenes.stop}_{reference_scenes_aggregation_technique}_{resolution}.parquet')

CPU times: user 3.55 s, sys: 313 ms, total: 3.86 s
Wall time: 3.48 s


In [1]:
# resolution = 100
# years = np.arange(2015,2024,1)
# reference_scenes_aggregation_technique = 'median' # mean / median / max

# for year in years:
#     print(f'Working on {year}...')
#     time_reference_scenes = slice(f'{year}-06-01',f'{year}-07-31') 
#     time_target_scene = f'{year}-04-27' # try and support time_wet_scene_asc and time_wet_scene_desc
    
    
#     dataset = create_dataset.create_dataset(bbox_gdf, 
#                                         year, 
#                                         time_reference_scenes, 
#                                         time_target_scene, 
#                                         reference_scenes_aggregation_technique, 
#                                         resolution)
    
    
#     dataset.to_zarr(f'data/datasets/{geojson.split(".")[0]}_{year}_tar_{time_target_scene}_ref_{time_reference_scenes.start}_{time_reference_scenes.stop}_{reference_scenes_aggregation_technique}_{resolution}.zarr',mode='w')
#     dataframe = create_dataset.dataset_to_dataframe(dataset)
#     dataframe.to_parquet(f'data/dataframes/{geojson.split(".")[0]}_{year}_tar_{time_target_scene}_ref_{time_reference_scenes.start}_{time_reference_scenes.stop}_{reference_scenes_aggregation_technique}_{resolution}.parquet')
